In [1]:
import pandas as pd
import numpy as np

In [2]:
import scipy.stats as st
import statsmodels.stats.multitest as smm

In [3]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')

In [4]:
df.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [5]:
df.shape

(72, 15750)

In [6]:
df.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [46]:
%%time
data_tstats = []
normal = df[df['Diagnosis'] == 'normal']
cancer = df[df['Diagnosis'] == 'cancer']
early_neoplasia = df[df['Diagnosis'] == 'early neoplasia']
    
for column in df.columns[2:]:
    t_stat, p_value = st.ttest_ind(normal[column], early_neoplasia[column], equal_var=False)
    data_tstats.append(['normal', 'early_neoplasia', column, t_stat, p_value])

    t_stat, p_value = st.ttest_ind(early_neoplasia[column], cancer[column], equal_var=False)
    data_tstats.append(['early_neoplasia', 'cancer', column, t_stat, p_value])

Wall time: 24.5 s


In [47]:
data_tstats = pd.DataFrame.from_records(data_tstats)
data_tstats.columns = ['group_1', 'group_2', 'gene', 't_stats', 'p']

In [48]:
data_tstats.head()

,group_1,group_2,gene,t_stats,p
0,normal,early_neoplasia,LOC643837,0.400289,0.690766
1,early_neoplasia,cancer,LOC643837,0.824849,0.413735
2,normal,early_neoplasia,LOC100130417,-4.608766,0.000032
3,early_neoplasia,cancer,LOC100130417,0.452236,0.653429
4,normal,early_neoplasia,SAMD11,-1.929277,0.060273


In [49]:
data_tstats.shape

(31496, 5)

In [50]:
data_neoplasia = data_tstats.loc[data_tstats['group_2'] == 'early_neoplasia']
data_cancer = data_tstats.loc[data_tstats['group_2'] == 'cancer']

In [51]:
data_neoplasia

,group_1,group_2,gene,t_stats,p
0,normal,early_neoplasia,LOC643837,0.400289,0.690766
2,normal,early_neoplasia,LOC100130417,-4.608766,0.000032
4,normal,early_neoplasia,SAMD11,-1.929277,0.060273
6,normal,early_neoplasia,NOC2L,0.220542,0.826429
8,normal,early_neoplasia,KLHL17,-2.013201,0.049876
...,...,...,...,...,...
31486,normal,early_neoplasia,DDX3Y,-0.470046,0.640904
31488,normal,early_neoplasia,CD24,-0.215891,0.830134
31490,normal,early_neoplasia,CYorf15B,-0.428747,0.670395
31492,normal,early_neoplasia,KDM5D,-0.262910,0.793925


In [52]:
data_neoplasia[data_neoplasia['p'] < 0.05]

,group_1,group_2,gene,t_stats,p
2,normal,early_neoplasia,LOC100130417,-4.608766,0.000032
8,normal,early_neoplasia,KLHL17,-2.013201,0.049876
14,normal,early_neoplasia,HES4,-2.337789,0.023712
18,normal,early_neoplasia,AGRN,-2.138607,0.037933
20,normal,early_neoplasia,C1orf159,-2.412757,0.020002
...,...,...,...,...,...
31374,normal,early_neoplasia,ZNF185,-2.083196,0.043227
31400,normal,early_neoplasia,SRPK3,2.151725,0.036647
31452,normal,early_neoplasia,SLC10A3,2.801084,0.007540
31456,normal,early_neoplasia,G6PD,-2.224274,0.032166


In [53]:
data_neoplasia[data_neoplasia['p'] <= 0.05].shape

(1575, 5)

In [54]:
data_cancer[data_cancer['p'] < 0.05].shape

(3490, 5)

In [55]:
reject, p_corrected, a1, a2 = smm.multipletests(data_neoplasia['p'], alpha=0.05/2, method='holm')
data_neoplasia['Holm_binar'] = reject
data_neoplasia['Holm_p'] = p_corrected

c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [130]:
smm.multipletests?

In [56]:
data_neoplasia['Holm_binar'].sum()

2

In [57]:
data_neoplasia[data_neoplasia['Holm_binar'] == True]

,group_1,group_2,gene,t_stats,p,Holm_binar,Holm_p
14488,normal,early_neoplasia,PCSK4,-5.806801,7.955435e-07,True,0.012527
19640,normal,early_neoplasia,EEF1A2,-6.524922,8.498742e-08,True,0.001338


In [58]:
holm = smm.multipletests(data_cancer['p'], alpha=0.05/2, method='holm')
data_cancer['Holm_binar'], data_cancer['Holm_p'] = holm[:2]

c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
data_cancer[data_cancer['Holm_binar']].sort_values('Holm_p', ascending=False)

,group_1,group_2,gene,t_stats,p,Holm_binar,Holm_p
22645,early_neoplasia,cancer,TACC3,-5.728412,1.484616e-06,True,0.023264
24717,early_neoplasia,cancer,LOC728264,5.824061,1.315217e-06,True,0.020611
24585,early_neoplasia,cancer,PSD2,5.806020,1.133485e-06,True,0.017764
2477,early_neoplasia,cancer,FMO2,5.751625,1.103311e-06,True,0.017292
27747,early_neoplasia,cancer,AASS,5.623122,1.061259e-06,True,0.016634
...,...,...,...,...,...,...,...
30181,early_neoplasia,cancer,LAMC3,8.424379,1.938708e-09,True,0.000031
15793,early_neoplasia,cancer,PRX,8.083884,1.416453e-09,True,0.000022
19573,early_neoplasia,cancer,EDN3,7.764647,7.855048e-10,True,0.000012
4577,early_neoplasia,cancer,SYT8,8.698478,1.398801e-10,True,0.000002


In [60]:
data_cancer['Holm_binar'].sum()

79

In [61]:
gene_neoplasia = data_neoplasia['gene'][data_neoplasia['Holm_binar']]
gene_cancer = data_cancer['gene'][data_cancer['Holm_binar']]

In [62]:
def fold_change(T, C):
    df = pd.DataFrame(data={'T_mean': T.mean(), 'C_mean': C.mean(), 'T>C': T.mean()>C.mean()})
    df1 = df[df['T>C']]
    df1['fold_change'] = df1['T_mean']/df1['C_mean']
    df2 = df[df['T>C'] == False]
    df2['fold_change'] = -df2['C_mean']/df2['T_mean']
    df = df1.append(df2)
    return df

In [63]:
fold_change_neoplasia = fold_change(early_neoplasia[gene_neoplasia], normal[gene_neoplasia])
fold_change_neoplasia[abs(fold_change_neoplasia['fold_change']) >= 1.5]

,T_mean,C_mean,T>C,fold_change
PCSK4,22.621756,14.983424,True,1.509785
EEF1A2,38.260320,19.373612,True,1.974868


In [64]:
fold_change_cancer = fold_change(cancer[gene_cancer], normal[gene_cancer])
fold_change_cancer[abs(fold_change_cancer['fold_change']) >= 1.5].shape

c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(73, 4)

In [65]:
fold_change_cancer[abs(fold_change_cancer['fold_change']) >= 1.5]

,T_mean,C_mean,T>C,fold_change
GABRD,13.320553,5.506915,True,2.418878
DPEP1,14.124519,6.936506,True,2.036258
TACC3,17.984323,11.898540,True,1.511473
INHBA,24.797881,7.908129,True,3.135746
EXTL1,9.749268,18.942555,False,-1.942972
...,...,...,...,...
LAMC3,14.175370,34.696688,False,-2.447674
SLC7A3,2.962692,8.225993,False,-2.776526
CAPN6,3.935252,11.435708,False,-2.905965
GPC3,5.496242,13.216064,False,-2.404564


In [66]:
reject, p_corrected = smm.fdrcorrection(data_neoplasia['p'], alpha=0.025)
data_neoplasia['BH_reject'] = reject
data_neoplasia['BH_p'] = p_corrected

c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
reject, p_corrected = smm.fdrcorrection(data_cancer['p'], alpha=0.025)
data_cancer['BH_reject'] = reject
data_cancer['BH_p'] = p_corrected

c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
gene2_neoplasia = data_neoplasia['gene'][data_neoplasia['BH_reject']]
gene2_cancer = data_cancer['gene'][data_cancer['BH_reject']]

In [69]:
fold_change_neoplasia_BH = fold_change(early_neoplasia[gene2_neoplasia], normal[gene2_neoplasia])
fold_change_neoplasia_BH[abs(fold_change_neoplasia_BH['fold_change']) >= 1.5]

,T_mean,C_mean,T>C,fold_change
TMEM63C,30.010954,18.037315,True,1.663826
PCSK4,22.621756,14.983424,True,1.509785
CACNG8,5.849340,3.333050,True,1.754951
EEF1A2,38.260320,19.373612,True,1.974868


In [70]:
fold_change_cancer_BH = fold_change(cancer[gene2_cancer], early_neoplasia[gene2_cancer])
fold_change_cancer_BH[abs(fold_change_cancer_BH['fold_change']) > 1.5].shape

c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\cohan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(524, 4)

In [71]:
fold_change_cancer_BH[abs(fold_change_cancer_BH['fold_change']) > 1.5]

,T_mean,C_mean,T>C,fold_change
ISG15,91.706112,37.296084,True,2.458867
GABRD,13.320553,7.224275,True,1.843860
MFAP2,4.863978,2.555026,True,1.903690
AIM1L,7.177703,3.627885,True,1.978481
CLSPN,5.881522,2.820314,True,2.085414
...,...,...,...,...
HS6ST2,2.582647,5.815704,False,-2.251839
GPC3,5.496242,10.280356,False,-1.870434
NCRNA00086,2.271824,3.670774,False,-1.615783
DUSP9,3.830095,8.324675,False,-2.173490
